In [1]:
import os
import pathlib

In [2]:
if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    phase = "test"
else:
#     phase = "train"  # calc CV on only 'validation'
    phase = "dev"  # to check if submission can run as same as 'test', but n_series_ids is larger ~270, in 'dev'.
#     phase = "test"


downsample_rate = 2
# score_th = 0.005
# distance = 96
score_th = 1e-4
distance = 88

# n_records_per_series_id = None
n_records_per_series_id = 1000


inference_step_offsets = None
# inference_step_offsets = np.arange(0, 24, 4) * 12 * 60

kaggle_model_dir_path = pathlib.Path("../cmi-dss-ensemble-models")
# kaggle_model_dir_path = pathlib.Path("/kaggle/input/cmi-dss-ensemble-models")
predicted_dir_path = pathlib.Path("/kaggle/working/predicted")

all_model_dir_path_dict = {
     3: (f"{kaggle_model_dir_path}/jumtras/exp016-gru-feature-fp16-layer4-ep70-lr-half", 64),
     7: (f"{kaggle_model_dir_path}/ranchantan/exp015-lstm-feature-108-sigma", 64),
    19: (f"{kaggle_model_dir_path}/ranchantan/exp019-stacked-gru-4-layers-24h-duration-4bs-108sigma", 16),
    27: (f"{kaggle_model_dir_path}/jumtras/exp027-TimesNetFeatureExtractor-1DUnet-Unet", 32),
    41: (f"{kaggle_model_dir_path}/ranchantan/exp041_retry", 32),
    47: (f"{kaggle_model_dir_path}/ranchantan/exp047_retry", 16),
#     50: (f"{kaggle_model_dir_path}/ranchantan/exp050-transformer-decoder_retry", 16),    
    50: (f"{kaggle_model_dir_path}/ranchantan/exp050-transformer-decoder_retry_resume", 16),
    52: (f"{kaggle_model_dir_path}/jumtras/exp052", 32),
    53: (f"{kaggle_model_dir_path}/jumtras/exp053", 16),
    55: (f"{kaggle_model_dir_path}/ranchantan/exp055", 16),
    58: (f"{kaggle_model_dir_path}/jumtras/exp058", 16),
    85: (f"{kaggle_model_dir_path}/jumtras/exp085", 16),
    88: (f"{kaggle_model_dir_path}/jumtras/exp088", 16)
}

weight_dict = {
    "47": 0,
    "50": 0,
    "52": 0,
    "53": 0,
}  # stacking

# weight_dict = {
#   "19": 0.1,
#   "50": 0.2,
#   "53": 0.3,
#   "58": 0.1,
#   "85": 0.2,
#   "88": 0.1
# }  # 24

# weight_dict = {
#     "19": 0.05,
#     "27": 0.05,
#     "50": 0.2,
#     "53": 0.3,
#     "58": 0.1,
#     "85": 0.2,
#     "88": 0.1,
# }  # 25

post_process_modes = {
#     "sleeping_edges_as_probs": cmi_dss_lib.utils.post_process.SleepingEdgesAsProbsSetting(
#         sleep_prob_th=0.2, min_sleeping_hours=6
#     ),
#     "cutting_probs_by_sleep_prob": cmi_dss_lib.utils.post_process.CuttingProbsBySleepProbSetting(
#         watch_interval_hour=7.5, sleep_occupancy_th=0.03
#     ),
#     "cutting_probs_by_sleep_prob": dict(
#         onset=cmi_dss_lib.utils.post_process.CuttingProbsBySleepProbSetting(
#             watch_interval_hour=7.5,
#             sleep_occupancy_th=0.04,
#         ),
#         wakeup=cmi_dss_lib.utils.post_process.CuttingProbsBySleepProbSetting(
#             watch_interval_hour=6.0,
#             sleep_occupancy_th=0.03,
#         ),
#     )    
}


# Validation

for p, _ in all_model_dir_path_dict.values():
    assert pathlib.Path(p).exists(), p

model_dir_path_info_dict = {
    all_model_dir_path_dict[int(exp_id)][0]: (weight, all_model_dir_path_dict[int(exp_id)][1])
    for exp_id, weight in weight_dict.items()
}

In [5]:
# use_stacking = True
use_stacking = False
stacking_exp_name = "s_exp006"

if use_stacking:
    stacking_model_dir_path = pathlib.Path("/kaggle/input/cmi-dss-seg-stacking-model") / stacking_exp_name
    assert stacking_model_dir_path.exists()
else:
    stacking_model_dir_path = None

In [6]:
import cmi_dss_lib.submission.inference

if use_stacking:
    assert inference_step_offsets is None
    for i_fold in range(5):
        for p, (_, batch_size) in model_dir_path_info_dict.items():
            model_dir_path = kaggle_model_dir_path / p
            !python -m run.inference {model_dir_path} kaggle.yaml batch_size={batch_size} --folds {i_fold}
        !python -m run.stacking.inference {stacking_model_dir_path} kaggle.yaml batch_size=32 --folds {i_fold} && \
         rm -rf /kaggle/working/predicted/ranchantan /kaggle/working/predicted/jumtras
    cmi_dss_lib.submission.inference.replace_predicted_pred_dirs_with_mean_preds_dir(predicted_dir_path, stacking_model_dir_path, phase)
else:
    for p, (_, batch_size) in model_dir_path_info_dict.items():
        model_dir_path = kaggle_model_dir_path / p
        for i_fold in range(5):
            if inference_step_offsets is None:
                !python -m run.inference {model_dir_path} kaggle.yaml batch_size={batch_size} --folds {i_fold}
            else:
                for inference_step_offset in inference_step_offsets:
                    !python -m run.inference --folds {i_fold} {model_dir_path} kaggle.yaml batch_size={batch_size} inference_step_offset={inference_step_offset}
                cmi_dss_lib.submission.inference.save_overlapping_mean_as_npz(
                    predicted_dir_path, 
                    model_dir_path,
                    i_fold,
                    phase,
                    inference_step_offsets
                )

        if phase in ["test", "dev"]:
            cmi_dss_lib.submission.inference.replace_predicted_pred_dirs_with_mean_preds_dir(
                predicted_dir_path, 
                model_dir_path,
                phase
            )

TypeError: 'type' object is not subscriptable